In [1]:
%load_ext bigdata
%hive_start
%timeout 300

In [2]:
!hdfs dfs -rm -r -f /tmp/output

Deleted /tmp/output


In [3]:
!hdfs dfs -mkdir /tmp/output

In [4]:
!hdfs dfs -copyFromLocal tbl0.csv /tmp/output/

In [5]:
%%hive

DROP TABLE IF EXISTS tbl0;
CREATE TABLE tbl0 (
    c1 INT,
    c2 STRING,
    c3 INT,
    c4 DATE,
    c5 ARRAY<CHAR(1)>,
    c6 MAP<STRING, INT>
)
ROW FORMAT DELIMITED
FIELDS TERMINATED BY ','
COLLECTION ITEMS TERMINATED BY ':'
MAP KEYS TERMINATED BY '#'
LINES TERMINATED BY '\n';

LOAD DATA LOCAL INPATH 'tbl0.csv' INTO TABLE tbl0;

DROP TABLE IF EXISTS tbl0;
OK
Time taken: 9.57 seconds
CREATE TABLE tbl0 (
    c1 INT,
    c2 STRING,
    c3 INT,
    c4 DATE,
    c5 ARRAY<CHAR(1)>,
    c6 MAP<STRING, INT>
)
ROW FORMAT DELIMITED
FIELDS TERMINATED BY ','
COLLECTION ITEMS TERMINATED BY ':'
MAP KEYS TERMINATED BY '#'
LINES TERMINATED BY '\n';
OK
Time taken: 0.801 seconds
LOAD DATA LOCAL INPATH 'tbl0.csv' INTO TABLE tbl0;
Loading data to table default.tbl0
OK
Time taken: 1.31 seconds


In [6]:
%%hive

DROP TABLE IF EXISTS resultado;

CREATE TABLE resultado
AS 
SELECT  c2,
        collect_list(c1)
FROM tbl0
GROUP BY c2
;

INSERT OVERWRITE DIRECTORY '/tmp/output'
ROW FORMAT DELIMITED FIELDS TERMINATED BY ','
COLLECTION ITEMS TERMINATED BY ':'
SELECT * FROM resultado;

DROP TABLE IF EXISTS resultado;
OK
Time taken: 0.026 seconds
CREATE TABLE resultado
AS 
SELECT  c2,
        collect_list(c1)
FROM tbl0
GROUP BY c2
;
Query ID = root_20200118231951_51e37f65-79a0-4397-8ce6-f265ef2d7fec
Total jobs = 1
Launching Job 1 out of 1
Number of reduce tasks not specified. Estimated from input data size: 1
In order to change the average load for a reducer (in bytes):
  set hive.exec.reducers.bytes.per.reducer=<number>
In order to limit the maximum number of reducers:
  set hive.exec.reducers.max=<number>
In order to set a constant number of reducers:
  set mapreduce.job.reduces=<number>
Starting Job = job_1579385130637_0008, Tracking URL = http://282e0d2d9a5d:8088/proxy/application_1579385130637_0008/
Kill Command = /usr/local/hadoop/bin/hadoop job  -kill job_1579385130637_0008
Hadoop job information for Stage-1: number of mappers: 1; number of reducers: 1
2020-01-18 23:20:04,702 Stage-1 map = 0%,  reduce = 0%
2020-01-18 23:20:10,221 Stage-1 map = 100%,  reduce = 0

In [7]:
!hdfs dfs -copyToLocal /tmp/output output;

In [8]:
%%hive
DROP TABLE resultado;

DROP TABLE resultado;
OK
Time taken: 0.208 seconds


In [9]:
%hive_quit